In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import pickle
import sys
sys.path.append("../../../Benchmark/")
from benchmarker import Benchmarker

In [ ]:
import os
os.chdir("../../../datatset/Mouse_brain_sag/")

Prepare data

In [3]:
files = ["Anterior", "Posterior"]
adatas = []
for i in files:
    adata = sc.read_10x_mtx(i+"/filtered_feature_bc_matrix")
    spatial = pd.read_csv(i+"/spatial/tissue_positions_list.csv", index_col=0, header=None)
    spatial = spatial.reindex(adata.obs_names)
    adata.obsm["spatial"] = np.array(spatial.iloc[:, 3:5])
    adatas.append(adata)
# metas = [pd.read_csv(f"{i}/metadata.tsv", sep="\t") for i in files]
adatas[0].obs["batch"] = "left"
adatas[1].obs["batch"] = "right"

In [4]:
adatas

[AnnData object with n_obs × n_vars = 2696 × 31053
     obs: 'batch'
     var: 'gene_ids', 'feature_types'
     obsm: 'spatial',
 AnnData object with n_obs × n_vars = 3353 × 31053
     obs: 'batch'
     var: 'gene_ids', 'feature_types'
     obsm: 'spatial']

In [5]:
2696+3353

6049

In [ ]:
from benchmarker import transform_coord
import numpy as np
spatial = [ad.obsm["spatial"] for ad in adatas]
# spatial[0] = spatial[0] * 0.45
# spatial[1] = spatial[1] * 1
spatial = transform_coord(spatial, axis="x", align_mode="mid", margin_size=0, angle=90, horizontal=True)
for i, ad in enumerate(adatas):
    ad.obs_names = [j+f"_{i}" for j in ad.obs_names]
    ad.obsm["spatial"] = spatial[i]

In [43]:
# concat = sc.concat(adatas)
# concat.obs["spatial1"] =  list(concat.obsm["spatial"][:,0])
# concat.obs["spatial2"] =  list(concat.obsm["spatial"][:,1])

In [44]:
# concat.obs_names_make_unique()
# concat.var_names_make_unique()

In [45]:
# concat.write("concat.h5ad")

In [46]:
bm = Benchmarker(R_conda_env="Rbase")

In [47]:
# bm.h5ad2rds(in_file=concat, out_file="concat.rds", verbose=True)

Run all method

In [ ]:
# bm.run(RDS_file_path="concat.rds", H5AD_file_path="concat.h5ad",
#        save_path="../../../result/Mouse_brain_sag/embed/", 
#        n_cluster=30, verbose=True, workers=4, methods=["STAMP"])

Evaluate

In [51]:
adata = sc.read_h5ad("concat.h5ad")

In [ ]:
res_dict = bm.read_result("../../result/Mouse_brain_sag/embed/", 
                          index=list(adata.obs_names), 
                          save="../../result/Mouse_brain_sag/embed_dict.pkl",
                          methods=bm.all_methods + ["STADIA"])

In [53]:
# with open("../../result/DLPFC/diff_sample/embed_dict.pkl", "rb") as f:
#     res_dict = pickle.load(f)

In [54]:
adata.obs["cell_type"] = "Unknown"

In [55]:
# metrics = bm.cal_metrics(adata=adata, batch_key="batch", label_key="cell_type",
#                          res_dict=res_dict, methods="all", verbose=True, rep=1,
#                          min_max_scale=False, save="../../result/Mouse_brain_sag/metrics.pkl")

In [56]:
# with open("../../result/DLPFC/diff_sample/metrics.pkl", "rb") as f:
#     metrics = pickle.load(f)

In [57]:
# metric = metrics[0].drop(['Isolated labels', 'NMI', 'ARI', 'Silhouette label',
#        'cLISI', 'Bio conservation', 'Total'], axis=1)

In [ ]:
bm.set_plot_params(params_dict={"figure.dpi": 300}, font_file_path="./Helvetica.ttf")

2025-09-01 19:05:16 - INFO - Custom font 'Helvetica' has been set


In [ ]:
save_dir = "../../../figures/Mouse_brain_sag/"

In [60]:
# bm.plot_legend(category_lst=["Anterior", "Posterior"], marker="o", ncol=1,
#                save=f"{save_dir}/batch_legend.pdf")

In [61]:
# bm.plot_legend(category_lst=list(range(30)), marker="o", ncol=5,
#                save=f"{save_dir}/cluster_legend.pdf")

In [62]:
# bm.plot_heatmap(metric_df=metric,  total_name="Batch correction",
#                  save=f"{save_dir}/summary_heatmap.pdf")

In [63]:
# marker_score = bm.cal_marker_score(adata, label_dict=res_dict["Cluster"], 
#                                    marker_genes=[
#                                        "Cux2", "Rasgrf2", "Rorb", "Rspo1", "Etv1", "Fezf2", "Rbp4", "Bmp3", "Tbr1", "Cplx3", "Nxph4", # CTX
#                                        "Hpca", "C1ql2", # HP
#                                        "Ppp1r1b", "Gpr88",# Striatum
#                                        "Tcf7l2", "Rgs16", # Thalamus
#                                        "Fat2", "Gabra6", "Cbln3", "Neurod1", # CB
#                                        "Car8", "Gdf10", "Ppp1r17", # CB layer
#                                        "Gad1", "Meis2", "Omp", "Nefh","Doc2g", "Cdhr1", # OB
#                                        "Mbp", "Anln", "Cldn11", # FT
#                                        "Ptgds","Myoc", # Meninges
#                                        ],
#                                    re_run=True)

In [64]:
# bm.plot_marker_score(metric_df=marker_score, xlabel=None,
#                      save=f"{save_dir}/marker_score.pdf")

In [65]:
from benchmarker import split_adata, transform_coord
import numpy as np
spatial = [adata.obsm["spatial"]]
spatial = transform_coord(spatial, vertical=True)

In [66]:
# bm.plot_gene_exp(
#     adata=adata,
#     genes=["Cux2", # L2/3
#            "Rorb", # L4
#            "Fezf2", # L5
#            "Tbr1", # L6
#            "Cplx3", # L6b
#            "Hpca",
#            "C1ql2", # HP
#            "Ppp1r1b",# Striatum
#            "Tcf7l2", # Thalamus
#            "Cbln3", # CB
#            "Ppp1r17", # CB layer
#            "Meis2", "Cdhr1", "Omp",  # OB
#            "Mbp", # FT
#            "Ptgds", # Meninges
#            ],
#     embed=spatial,
#     figsize=(15, 3.), 
#     frameon=True, inner_gs_row=1, inner_gs_col=1, ncol=8,
#     xlabel=None, ylabel=None, only_show_left=True,
#     axis_width = 1.2, axis_color="lightgrey",
#     inner_col_wspace = -0.1,
#     outer_row_hspace=1.0, outer_col_wspace=0.05,
#     xlabel_pad=0.015,
#     sizes=[3,],
#     vmax="p99",
#     vmin="p30",
#     save=f"{save_dir}/marker_gene_plot.pdf")

In [67]:
adata

AnnData object with n_obs × n_vars = 6049 × 31053
    obs: 'batch', 'spatial1', 'spatial2', 'cell_type'
    obsm: 'spatial'

In [68]:
bg_dict = {i:"#D4B483" if i in bm.spatial_methods else "#5873a4" for i in bm.all_methods }
bg_dict["STADIA"] = "#D4B483"

In [69]:
for key in res_dict["Cluster"].keys():
    print(key, len(set(res_dict["Cluster"][key].flatten())))

BASS 30
DeepST 30
Graspot 30
Harmony 30
LIGER 30
PRECAST 30
SEDR 30
SLAT 30
SPIRAL 30
STAMP 29
STAligner 30
Scanorama 30
Seurat 30
Unintegrated 30
scVI 30
spaVAE 30
STADIA 30


In [72]:
# bm.plot_spatial(spatial=spatial, label_dict=res_dict["Cluster"],
#                 figsize=(14, 12), frameon=True, inner_gs_row=1, inner_gs_col=1, size=10, ncol=4,
#                 xlabel="name", ylabel=None, only_show_left=True,
#                 axis_width = 1.2, axis_color="lightgrey",
#                 # order=["STADIA", "SPIRAL", "STAMP", "SEDR", "Seurat", "Harmony", "scVI", "Scanorama"],
#                 outer_row_hspace=0.3, outer_col_wspace=0.01, inner_common_camp=False,
#                 background_color = lambda x: bg_dict[x] if x in bg_dict else None,
#                 xlabel_pad=0.015, save_dpi=600,
#                 save=f"{save_dir}/all_spatial.png")

In [73]:
# bm.plot_umap(embed_dict=res_dict["UMAP"], batch_dict=res_dict["Batch"], annot_list=None, 
#              figsize=(14, 7), frameon=True, inner_gs_row=1, inner_gs_col=1, size=2, ncol=6,
#              xlabel="name", only_show_top=False, only_show_left=True,
#              background_color = lambda x: bg_dict[x] if x in bg_dict else None,
#             #  order=["Unintegrated", "STADIA", "SPIRAL", "STAMP", "SEDR", "Seurat", "Harmony", "scVI", "Scanorama"],
#              axis_width=1.2, axis_color="lightgrey",
#              outer_col_wspace=0.05, outer_row_hspace=0.3, save_dpi=600,
#              ylabel_pad=0.02,xlabel_pad=0.012, merge=False, merge_margin_size=0.4,
#              save=f"{save_dir}/all_umap.png")